## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabweichung für die aufgenommenen Messwerte aus der Datei "merkmaleRoh.csv" der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit einer Länge von 80ms (siehe hierzu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung). Bei dem Beschleunigungssensor sind dies die Messwerte der beschleunigung von den 3-Achsen X, Y sowie Z (Abb. 1). Des Weiteren werden die Gier-Nick-Roll Winkel in rad in einem Zeitfenster erfasst. Für die Berechnung des Mittelwerts, Varianz und Standardabweichung über die Messwerte wird die Form der abgespeicherten Messwerte geändert, weil diese als Python Objekte verwaltet werden mit welchen nicht gerechnet werden kann. Zu den Messwertreihen wird jeweils eine CSV Datei erstellt. Eine Reihe stellt dann in dieser Datei eine Messreihe zu einem Zeitfenster dar. Bei der ausführung dieses Notebooks werden deshalb zusätzlich folgende CSV Dateien erstellt:

X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2  
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad  
Nick Messwerte in rad  
Roll Messwerte in rad   
Zeitstempel der Messwerte in ns  

<img src="../daten/abbildungen/sensorOrientierung.png" alt="Android Smartphone Sensor Orientierung" width="200px" />
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: siehe Bildverzeichnis Abb. 2 in der README.md Datei

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [2]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_26_bis_27/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit
count,4.963000e+03,4963.000000,4963.000000,4963.000000,4963.000000,4.963000e+03,4.963000e+03,4.963000e+03,4.963000e+03,4.963000e+03,4.963000e+03
mean,1.525693e+12,53.583963,10.084546,15.184848,62.252871,1.527235e+13,1.527227e+13,1.527260e+13,9.923256e+06,1.527309e+13,1.525693e+12
std,1.231895e+07,0.016691,0.035909,4.105801,0.448394,9.408304e+12,9.408303e+12,9.408305e+12,9.941265e+06,9.408308e+12,1.231896e+07
min,1.525679e+12,53.556515,10.020442,5.004000,60.000000,4.720012e+12,4.719938e+12,4.720260e+12,7.630070e+05,4.720751e+12,1.525679e+12
25%,1.525681e+12,53.571229,10.054037,12.492000,62.000000,6.161229e+12,6.161143e+12,6.161476e+12,2.298216e+06,6.161965e+12,1.525681e+12
50%,1.525682e+12,53.579170,10.094249,15.624002,62.000000,7.806778e+12,7.806702e+12,7.807027e+12,5.331862e+06,7.807517e+12,1.525682e+12
75%,1.525705e+12,53.597912,10.116828,18.108002,63.000000,2.491194e+13,2.491186e+13,2.491218e+13,1.000184e+07,2.491267e+13,1.525705e+12
max,1.525707e+12,53.613850,10.137879,24.948000,67.000000,2.659563e+13,2.659555e+13,2.659587e+13,2.999632e+07,2.659633e+13,1.525707e+12


In [3]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4963 entries, 0 to 4962
Data columns (total 18 columns):
Zeitstempel in Unixzeit                                               4963 non-null int64
Breitengrad                                                           4963 non-null float64
Laengengrad                                                           4963 non-null float64
Geschwindigkeit in km/h                                               4963 non-null float64
X-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Z-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Gier Messwerte in rad                                                 4963 non-null object
Nick Messwerte in rad                                                 4963 non-null object
Roll Messwerte in rad                                                 49

In [208]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

0    0.66929 0.81137 0.75647 0.22221 -0.14389 -0.13...
1    0.2285 0.13695 0.37016 0.08172 -0.05709 0.3834...
2    -2.72057 3.72297 3.37689 -1.28232 -0.90316 0.4...
3    -0.06289 0.54739 0.6219 -1.35643 -0.54872 0.28...
4    -0.5752 -1.41006 -1.3118 -0.58965 -1.32969 0.6...
Name: X-Achse Beschleunigungswerte in m/s^2, dtype: object

In [209]:
# Die Anzahl der Messwerte sollte immer gleich sein, ansonsten werden fehlende Spalten mit NaN Werten gefüllt bei der
# späteren Abspeicherung der erfassten Messwerte wie z.B. in die Datei x-Achse-Beschleunigungswerte.csv.
# Um die Berechnung von den Merkmalen Varianz und Mittelwert unteranderem zu beschleunigen ist es wichtig sogenannte Außenseiter
# aus den Datensätzen zu entfernen.
# Dafür muss auch das Bild zu einer Reihe aus der merkmaleRoh.csv Datei gelöscht werden.
# Dies wird erreicht indem alle Pfade der Bilder geladen werden.
# Dann können Reihen sowie Bilder gelöscht werden mit mehr Messwerten als ein bestimmter Grenzwert.
print(featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23])

################### Entferne Außenseiter mit Anzahl der Messwerte > Grenzwert ###################
#### Warnung löschen der zugeschnittennen Bilder kann nicht rückgängig gemacht werden!
#threshold  = 23 
#imagePaths = []
# Hier können die Datensätze ausgewählt werden
#datasets = ['15','16','17','18','19','20','21','22']
# Die Pfade zu den Ordnern in welchem sich die Bilder befinden
#paths = []
#for dataset in datasets: # Für jeden Datensatz merke Pfad
#    paths.append("C:/Users/morro/Documents/datenRoh/" + dataset + "/zugeschnitten/")
#for path in paths: # Für jeden Pfad hole die Namen der Ordner
#    folders = os.listdir(path)
#    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
#        filesPath = path + folder + "/"
#        files = os.listdir(filesPath)
#        for name in files: # Ein Dateiname aus diesem Ordner
#            if "jpg" not in name:
#                continue
#            imagePaths.append(filesPath + name) # Path wird geladen in Liste
#toRemoveImagesIndex = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > threshold].index.tolist()
#for i in toRemoveImagesIndex:
#    print(imagePaths[i])
#    os.remove(imagePaths[i])
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]
featuresDf.drop(unwantedRows.index, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

14959    24
35516    48
35525    28
45468    47
52621    24
54376    24
54719    24
55901    40
56033    24
59055    24
59514    24
59990    38
60418    24
62192    25
64457    28
64506    24
68134    24
80666    24
94258    27
95849    24
97012    43
Name: Anzahl der Messwerte, dtype: int64


In [4]:
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]

0       62
1       63
2       63
3       62
4       62
5       63
6       62
7       63
8       63
9       63
10      62
11      62
12      62
13      62
14      62
15      63
16      62
17      62
18      63
19      62
20      62
21      62
22      62
23      63
24      62
25      62
26      62
27      62
28      63
29      62
        ..
4933    62
4934    62
4935    62
4936    62
4937    62
4938    63
4939    62
4940    63
4941    63
4942    62
4943    63
4944    62
4945    62
4946    62
4947    62
4948    62
4949    62
4950    62
4951    62
4952    63
4953    62
4954    62
4955    62
4956    62
4957    62
4958    62
4959    62
4960    62
4961    62
4962    63
Name: Anzahl der Messwerte, Length: 4963, dtype: int64

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [5]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4963 entries, 0 to 4962
Data columns (total 18 columns):
Zeitstempel in Unixzeit                                               4963 non-null int64
Breitengrad                                                           4963 non-null float64
Laengengrad                                                           4963 non-null float64
Geschwindigkeit in km/h                                               4963 non-null float64
X-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Z-Achse Beschleunigungswerte in m/s^2                                 4963 non-null object
Gier Messwerte in rad                                                 4963 non-null object
Nick Messwerte in rad                                                 4963 non-null object
Roll Messwerte in rad                                                 49

In [6]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [7]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

array([[ ['-0.10848', '-0.14789', '-0.17942', '-0.32723', '-0.76725', '-0.44506', '0.24172', '0.4234', '0.32359', '0.74939', '0.90615', '0.54123', '0.29526', '0.19042', '-0.03135', '-1.26606', '-2.9127', '-3.08079', '-0.73297', '-1.2604', '1.50483', '3.10408', '0.58341', '-0.22262', '0.84872', '2.51791', '0.0685', '-1.89102', '-1.72715', '-0.70733', '-0.94875', '-0.45236', '-0.80607', '-0.50676', '0.57544', '1.68637', '1.74766', '0.96928', '-0.31231', '-1.30695', '-1.30586', '-1.22838', '-1.53415', '-1.31908', '-1.11637', '-0.90823', '-0.26671', '-0.24383', '0.28013', '0.65333', '1.01318', '1.80671', '1.78266', '1.57954', '1.14123', '1.35754', '0.42733', '0.60253', '0.43624', '-0.04921', '-0.33033', '-0.52456']],
       [ ['-0.53972', '-1.32033', '-0.76494', '-0.21338', '-1.30441', '-2.22321', '-0.73643', '0.57558', '0.78244', '-0.35453', '-0.3907', '0.83684', '1.08338', '1.23463', '0.86531', '0.35532', '0.08525', '0.57403', '0.79652', '0.68336', '0.11783', '0.09445', '0.30559', '0.045

In [9]:
accXList[12] # zeige Liste

['-0.65282',
 '-0.53739',
 '1.02595',
 '-0.42022',
 '-1.99086',
 '0.32286',
 '1.98996',
 '-1.1813',
 '-0.7763',
 '-1.99991',
 '-0.31262',
 '-2.14995',
 '-0.41734',
 '1.09134',
 '0.39826',
 '-1.36673',
 '-0.23512',
 '-1.99602',
 '-0.24822',
 '0.3992',
 '0.10503',
 '2.26005',
 '1.67032',
 '-0.62489',
 '0.0519',
 '-0.31073',
 '0.30323',
 '-0.24757',
 '1.80943',
 '0.43642',
 '-0.21763',
 '0.13254',
 '-0.95106',
 '0.22',
 '1.50927',
 '0.01241',
 '0.62303',
 '0.91233',
 '1.18974',
 '0.2318',
 '-1.02489',
 '-0.6052',
 '0.22087',
 '-0.63523',
 '-0.59994',
 '-0.37251',
 '0.26912',
 '-0.91841',
 '-0.93375',
 '-1.05327',
 '0.23017',
 '0.44481',
 '0.29474',
 '0.74164',
 '-0.11136',
 '-0.57924',
 '0.7473',
 '-0.52055',
 '2.03541',
 '0.14219',
 '-0.5143',
 '1.32022']

In [10]:
np.array(accXList[1]).dtype

dtype('<U8')

In [11]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(abs(np.array(accXList[0]).astype(float))).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())

In [12]:
accXDf = pandas.concat(accList, ignore_index=True)

In [13]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,0.10848,0.14789,0.17942,0.32723,0.76725,0.44506,0.24172,0.42340,0.32359,0.74939,...,0.60253,0.43624,0.04921,0.33033,0.52456,NaN,NaN,NaN,NaN,NaN
1,0.53972,1.32033,0.76494,0.21338,1.30441,2.22321,0.73643,0.57558,0.78244,0.35453,...,0.93840,1.45597,3.27951,1.82700,2.07470,0.75589,NaN,NaN,NaN,NaN
2,0.24503,0.42604,0.04989,0.37719,0.34162,0.67151,0.79751,0.83703,0.99122,0.42504,...,0.01050,0.10016,0.15656,0.69200,0.51918,0.53811,NaN,NaN,NaN,NaN
3,0.49485,0.00270,0.44671,0.28601,0.39961,0.19729,0.37829,0.84677,0.17195,0.50549,...,0.41299,0.39150,0.46624,0.14297,0.68260,NaN,NaN,NaN,NaN,NaN
4,0.07049,0.77675,1.23450,0.43616,0.43236,1.32637,1.16817,0.68919,0.53583,0.13736,...,1.23006,0.92294,0.45666,1.11596,0.33325,NaN,NaN,NaN,NaN,NaN


In [14]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.T.mean().head(20)

0     0.924603
1     0.783440
2     0.529428
3     0.491929
4     0.637369
5     0.776810
6     0.447416
7     0.628181
8     0.454039
9     0.658008
10    0.657937
11    0.985380
12    0.768498
13    0.820796
14    1.115941
15    1.037056
16    1.201934
17    0.981223
18    1.181004
19    0.722605
dtype: float64

In [15]:
accXDf.shape

(4963, 67)

In [16]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(abs(np.array(accYList[0]).astype(float))).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [17]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4963 entries, 0 to 4962
Data columns (total 67 columns):
0     4963 non-null float64
1     4963 non-null float64
2     4963 non-null float64
3     4963 non-null float64
4     4963 non-null float64
5     4963 non-null float64
6     4963 non-null float64
7     4963 non-null float64
8     4963 non-null float64
9     4963 non-null float64
10    4963 non-null float64
11    4963 non-null float64
12    4963 non-null float64
13    4963 non-null float64
14    4963 non-null float64
15    4963 non-null float64
16    4963 non-null float64
17    4963 non-null float64
18    4963 non-null float64
19    4963 non-null float64
20    4963 non-null float64
21    4963 non-null float64
22    4963 non-null float64
23    4963 non-null float64
24    4963 non-null float64
25    4963 non-null float64
26    4963 non-null float64
27    4963 non-null float64
28    4963 non-null float64
29    4963 non-null float64
30    4963 non-null float64
31    4963 non-null float

In [18]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66
0,1.37155,0.51516,0.55304,1.17775,1.84676,2.88729,0.06503,1.73737,0.74615,0.82658,0.74862,0.12110,0.98542,0.88010,1.76310,6.42124,1.06128,3.68636,4.77382,3.58939,...,0.37413,0.37574,0.12863,0.57112,0.13494,0.68902,1.05706,0.41065,1.37029,0.16136,0.80579,0.41498,0.29608,0.43588,0.24126,NaN,NaN,NaN,NaN,NaN
1,2.00218,0.22286,2.22722,1.27594,0.22058,0.94280,0.55522,0.76615,0.29096,0.19645,0.20332,0.00572,0.24956,0.06102,0.03368,0.19568,0.23334,0.05832,0.45918,0.78125,...,1.19159,2.66275,2.32921,0.74169,1.55888,0.86422,0.65686,0.20353,2.16644,4.95114,1.47878,2.15717,1.15514,0.81703,0.31154,1.18374,NaN,NaN,NaN,NaN
2,1.21942,0.68458,0.18765,0.30316,1.33591,1.17616,0.34601,0.35162,0.66455,1.09876,0.27003,0.64488,0.19393,1.46925,0.06557,2.24345,0.38487,1.07135,1.08875,1.33051,...,0.23612,0.49554,0.18210,0.49829,1.10284,1.26516,0.04533,1.69132,0.78629,2.02165,0.98776,1.02023,0.37882,0.21811,1.72537,0.61398,NaN,NaN,NaN,NaN
3,2.19566,0.40829,0.92967,1.20246,0.46289,0.61055,0.25877,0.97267,1.10676,0.22672,1.02896,1.40526,0.20465,1.18487,0.44630,1.42946,0.46918,0.66680,0.41103,1.01854,...,1.40134,0.90138,3.64760,0.69607,2.01757,1.39972,3.07851,0.86905,1.48080,1.44530,1.95412,2.03813,0.60683,0.36306,1.01182,NaN,NaN,NaN,NaN,NaN
4,0.20345,0.31580,0.48231,1.24375,0.10843,1.06759,0.85425,1.09731,0.51636,0.12177,0.10196,0.07147,0.97637,0.13846,0.54794,1.25028,0.68547,0.41065,1.98505,0.72978,...,0.27824,1.18693,0.04532,0.17845,0.02036,1.07146,0.01642,0.29314,0.82296,0.13756,0.44734,0.78709,0.03226,1.00666,0.22113,NaN,NaN,NaN,NaN,NaN


In [19]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [20]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [21]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [22]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [23]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [24]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXDf.T.mean()
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
featuresDf['Mittelwert Roll in rad'] = rollDf.T.mean()

In [25]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

0    0.924603
1    0.783440
Name: Mittelwert X-Achse Beschleunigung in m/s^2, dtype: float64

In [26]:
featuresDf.columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h', 'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2', 'Gier Messwerte in rad',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [27]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [28]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [29]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [30]:
featuresDf['Gier-Aenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [31]:
accXDf[14957:14961]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66


In [33]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].iloc[10]

0.65793677419354823

In [34]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [35]:
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'].head()

0    0.53950
1    0.36854
2    0.19949
3    0.09243
4    0.24041
Name: Varianz X-Achse Beschleunigung in m/s^2, dtype: float64

In [36]:
calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])

array([ 0.73451,  0.60707,  0.44664, ...,  0.51258,  0.39152,  1.79728])

In [37]:
# Berechne Standardabweichung
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [38]:
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'].head()

0    0.73451
1    0.60707
2    0.44664
3    0.30402
4    0.49032
Name: Standardabweichung X-Achse Beschleunigung in m/s^2, dtype: float64

In [39]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4963 entries, 0 to 4962
Data columns (total 67 columns):
0     4963 non-null float64
1     4963 non-null float64
2     4963 non-null float64
3     4963 non-null float64
4     4963 non-null float64
5     4963 non-null float64
6     4963 non-null float64
7     4963 non-null float64
8     4963 non-null float64
9     4963 non-null float64
10    4963 non-null float64
11    4963 non-null float64
12    4963 non-null float64
13    4963 non-null float64
14    4963 non-null float64
15    4963 non-null float64
16    4963 non-null float64
17    4963 non-null float64
18    4963 non-null float64
19    4963 non-null float64
20    4963 non-null float64
21    4963 non-null float64
22    4963 non-null float64
23    4963 non-null float64
24    4963 non-null float64
25    4963 non-null float64
26    4963 non-null float64
27    4963 non-null float64
28    4963 non-null float64
29    4963 non-null float64
30    4963 non-null float64
31    4963 non-null float

In [252]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.66929,0.81137,0.75647,0.22221,0.14389,0.13032,0.34931,0.26401,0.34900,0.41699,0.27218,0.38619,0.29352,0.46454,0.47877,0.38290,0.13767,0.11002,0.01936,0.27609,NaN,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,0.05709,0.38348,0.27536,0.63390,0.21317,0.05919,0.44563,0.78543,0.52098,1.10619,0.19531,0.73840,0.60594,0.54593,0.00775,0.26951,NaN,NaN,NaN
2,2.72057,3.72297,3.37689,1.28232,0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,2.50279,0.54644,0.81943,0.67742,0.06682,0.23722,0.45391,1.20600,1.94558,2.88967,NaN,NaN,NaN
3,0.06289,0.54739,0.62190,1.35643,0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,1.86127,NaN,NaN
4,0.57520,1.41006,1.31180,0.58965,1.32969,0.65253,1.15038,0.27477,0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,0.67322,1.87156,1.45116,0.73922,NaN,NaN,NaN


In [40]:
# Änderungen in CSV Datei schreiben
#featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/merkmale.csv')

In [202]:
pandas.read_csv("../daten/merkmale_datensatz_15_bis_22/merkmale.csv").columns

Index(['Zeitstempel in Unixzeit', 'Anzahl der Messwerte',
       'Belichtungszeit in ns', 'Breitengrad', 'Geschwindigkeit in km/h',
       'Gier Messwerte in rad', 'Hoehe in m ueber WGS 84', 'Laengengrad',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Start der Belichtung in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2',
       'Zeitstempel der Messwerte in ns',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad', 'Gier-Aenderung in rad',
    

In [41]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accXDf.index.names = ["index"]
accXDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/xAchseBeschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/nickMesswerte.csv')
rollDf.index.names = ["index"]
rollDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/rollMesswerte.csv')
azimuthDf.index.names = ["index"]
azimuthDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/azimuthMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/sensoreventZeitstempel.csv')

In [204]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.040s

OK
